In [27]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

# Criando novas variáveis 

In [28]:
# Supondo que bd_full seja o seu DataFrame
bd_full = pd.read_csv('insurance_novo.csv', delimiter=',')

bd_full.rename(columns={"age":"idade"
 ,"sex":"gênero"
 ,"bmi":"imc"
 ,"children":"filhos"
 ,"smoker":"fumante"
 ,"region":"região"
 ,"charges":"encargos"}, inplace=True)



# Faixas etárias
bd_full['fx_idade'] = np.where(bd_full['idade'] <= 30, 'jovem',
                               np.where((bd_full['idade'] > 30) & (bd_full['idade'] <= 50), 'adulto',
                                        np.where((bd_full['idade'] > 50) & (bd_full['idade'] <= 70), 'idoso',
                                                 'idoso_master')))
print(bd_full['fx_idade'].value_counts())

# Faixas de IMC
bd_full['imc_table'] = np.where(bd_full['imc'] <= 18.4, 'Abaixo do peso',
                                np.where((bd_full['imc'] >= 18.5) & (bd_full['imc'] <= 24.9), 'Peso normal',
                                         np.where((bd_full['imc'] >= 25) & (bd_full['imc'] <= 29.9), 'Acima do peso',
                                                  np.where((bd_full['imc'] >= 30) & (bd_full['imc'] <= 34.9), 'Obesidade I',
                                                           np.where((bd_full['imc'] >= 35) & (bd_full['imc'] <= 40), 'Obesidade II',
                                                                    'Obesidade III')))))
print(bd_full['imc_table'].value_counts())

# Variável indicando se a pessoa tem filhos
bd_full['tem_filho'] = np.where(bd_full['filhos'] > 0, 1, 0)
print(bd_full['tem_filho'].value_counts())


fx_idade
adulto    538
jovem     444
idoso     356
Name: count, dtype: int64
imc_table
Obesidade I       388
Acima do peso     377
Obesidade II      225
Peso normal       222
Obesidade III     106
Abaixo do peso     20
Name: count, dtype: int64
tem_filho
1    764
0    574
Name: count, dtype: int64


# Tratando variáveis categóricas como numéricas


In [29]:
# Variável de sexo 
bd_full['gênero2'] = np.where(bd_full['gênero'] == 'female', 2,1)
print(bd_full['gênero'].value_counts())

# Variável indicado se a pessoa é fumante
bd_full['fumante'] = np.where(bd_full['fumante'] == 'yes', 1, 9)
print(bd_full['fumante'].value_counts())

# Variável de região
bd_full['região2'] = np.where(bd_full['região'] == 'northeast', 1,
                             np.where(bd_full['região'] == 'northwest', 2,
                                      np.where(bd_full['região'] == 'southeast', 3, 4)))
print(bd_full['região2'].value_counts())


gênero
male      676
female    662
Name: count, dtype: int64
fumante
9    1064
1     274
Name: count, dtype: int64
região2
3    364
4    325
2    325
1    324
Name: count, dtype: int64


# Seleção de variáveis para análise de correlação

In [30]:
bd_corr = bd_full[['idade', 'imc', 'filhos', 'tem_filho', 'gênero2', 'região2']]
print(bd_corr.corr())

              idade       imc    filhos  tem_filho   gênero2   região2
idade      1.000000  0.109272  0.042469   0.047075  0.020856  0.002127
imc        0.109272  1.000000  0.012759   0.016082 -0.046371  0.157566
filhos     0.042469  0.012759  1.000000   0.787569 -0.017163  0.016569
tem_filho  0.047075  0.016082  0.787569   1.000000 -0.015111  0.013685
gênero2    0.020856 -0.046371 -0.017163  -0.015111  1.000000 -0.004588
região2    0.002127  0.157566  0.016569   0.013685 -0.004588  1.000000


# Análise Descritiva Univariada 

### Idade

In [31]:
print(bd_full['idade'].describe())
print(stats.describe(bd_full['idade']))



count    1338.000000
mean       39.207025
std        14.049960
min        18.000000
25%        27.000000
50%        39.000000
75%        51.000000
max        64.000000
Name: idade, dtype: float64
DescribeResult(nobs=1338, minmax=(18, 64), mean=39.20702541106129, variance=197.40138665754378, skewness=0.055610083072599126, kurtosis=-1.2449206804584227)


### Sexo

In [32]:
print(bd_full['gênero'].value_counts())

gênero
male      676
female    662
Name: count, dtype: int64


### IMC

In [33]:
print(bd_full['imc'].describe())
print(stats.describe(bd_full['imc']))

count    1338.000000
mean       30.663397
std         6.098187
min        15.960000
25%        26.296250
50%        30.400000
75%        34.693750
max        53.130000
Name: imc, dtype: float64
DescribeResult(nobs=1338, minmax=(15.96, 53.13), mean=30.66339686098655, variance=37.18788360977324, skewness=0.28372857291709386, kurtosis=-0.05502310583700032)


### Filhos

In [34]:
print(bd_full['filhos'].value_counts())

filhos
0    574
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64


### Fumantes

In [35]:
print(bd_full['fumante'].value_counts())

fumante
9    1064
1     274
Name: count, dtype: int64


### Região

In [36]:
print(bd_full['região'].value_counts())

região
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64


# Análise Descritiva Bivariada

### Filhos por região

In [37]:
print(bd_full[bd_full['tem_filho'] == 1]['região'].value_counts())

região
southeast    207
northwest    193
southwest    187
northeast    177
Name: count, dtype: int64


### Filhos por fx_idade

In [38]:
print(bd_full[bd_full['tem_filho'] == 1]['fx_idade'].value_counts())


fx_idade
adulto    406
jovem     196
idoso     162
Name: count, dtype: int64


### Filhos por gênero 

In [39]:
print(bd_full[bd_full['tem_filho'] == 1]['gênero'].value_counts())


gênero
male      391
female    373
Name: count, dtype: int64


### Filhos por fumantes

In [40]:
print(bd_full[bd_full['tem_filho'] == 1]['fumante'].value_counts())


fumante
9    605
1    159
Name: count, dtype: int64


### Fumantes por sexo

In [41]:
print(bd_full[bd_full['fumante'] == 'yes']['gênero'].value_counts())

Series([], Name: count, dtype: int64)


# Regressão Linear

### Funções 

In [42]:
# Função para treinar o modelo
def train_model(x, y):
    model = LinearRegression()
    model.fit(x,y)
    return model

# Função para avaliar modelo
def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    acc = 1 - np.sum(np.abs(y_pred - y_test)) / np.sum(y_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return acc * 100, mape * 100

### Dividir os dados em treino (70%) e teste (30%)

In [43]:
train_data, test_data = train_test_split(bd_full, test_size=0.3, random_state=42)

In [44]:
y_train = train_data['encargos']
x_train = train_data.drop('encargos', axis=1)
y_test = test_data['encargos']
x_test = test_data.drop('encargos', axis=1)

### Tratamento de variáveis

In [45]:
x_train['gênero2'] = np.where(x_train['gênero'] == 'female', 2, 1)
x_train['fuma'] = np.where(x_train['fumante'] == 'yes', 1, 0)
x_train['região2'] = np.where(x_train['região'] == 'northeast', 1, 
                              np.where(x_train['região'] == 'northwest', 2, 
                                       np.where(x_train['região'] == 'southeast', 3, 4)))

x_train['fx_idade'] = np.where(x_train['idade'] <= 30, 'jovem',
                              np.where((x_train['idade'] > 30) & (x_train['idade'] <= 30) & (x_train['idade'] <= 50), 'adulto',
                                       np.where((x_train['idade'] >50) & (x_train['idade'] <= 70), 'idoso', 
                                       'idoso_master')))





## Converter variáveis categóricas em fatores

In [46]:
# Criar dummies para as variáveis categóricas
x_train = pd.get_dummies(x_train, columns=['fx_idade', 'região', 'fumante', 'gênero', 'imc_table'], drop_first=True)
x_train_cols = x_train.columns

x_train

,idade,imc,filhos,tem_filho,gênero2,região2,fuma,fx_idade_idoso_master,fx_idade_jovem,região_northwest,região_southeast,região_southwest,fumante_9,gênero_male,imc_table_Acima do peso,imc_table_Obesidade I,imc_table_Obesidade II,imc_table_Obesidade III,imc_table_Peso normal
332,61,31.160,0,0,2,2,0,False,False,True,False,False,True,False,False,True,False,False,False
355,46,27.600,0,0,1,4,0,True,False,False,False,True,True,True,True,False,False,False,False
138,54,31.900,3,1,2,3,0,False,False,False,True,False,True,False,False,True,False,False,False
381,55,30.685,0,0,1,1,0,False,False,False,False,False,False,True,False,True,False,False,False
292,25,45.540,2,1,1,3,0,False,True,False,True,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,18,31.350,4,1,2,1,0,False,True,False,False,False,True,False,False,True,False,False,False
1130,39,23.870,5,1,2,3,0,True,False,False,True,False,True,False,False,False,False,False,True
1294,58,25.175,0,0,1,1,0,False,False,False,False,False,True,True,True,False,False,False,False
860,37,47.600,2,1,2,4,0,True,False,False,False,True,False,False,False,False,False,True,False


## Modelos

In [47]:
model1 = train_model(x_train[['idade', 'gênero2', 'imc', 'filhos', 'fuma', 'região2']], y_train)
model2 = train_model(x_train[['idade', 'gênero2', 'imc', 'fuma']], y_train)

## Categorizar a idade e treinar o modelo

In [48]:
model3 = train_model(x_train, y_train)

### Transformar os dados de teste da mesma maneira que os dados de treino

In [49]:
x_test['gênero2'] = np.where(x_test['gênero'] == 'female', 2, 1)
x_test['fuma'] = np.where(x_test['fumante'] == 'yes', 1, 0)
x_test['região2'] = np.where(x_test['região'] == 'northeast', 1,
                             np.where(x_test['região'] == 'northwest', 2,
                                      np.where(x_test['região'] == 'southeast', 3,4)))
x_test['fx_idade'] = np.where(x_test['idade'] <= 30, 'jovem',
                              np.where((x_test['idade'] > 30) & (x_test['idade'] <= 50), 'adulto',
                                       np.where((x_test['idade'] > 50) & (x_test['idade'] <= 70), 'idoso',
                                                'idoso_master')))


### Converter variáveis categóricas em dummies e reindexar para garantir que todas as colunas estejam presentes

In [50]:
x_test = pd.get_dummies(x_test, columns=['fx_idade', 'região', 'fumante', 'gênero', 'imc_table'], drop_first=True)
x_test = x_test.reindex(columns=x_train_cols, fill_value=0)

## Predições e avaliações

In [51]:
x_test_new1 = x_test[['idade', 'gênero2', 'imc', 'filhos', 'fuma', 'região2']]
acc1, mape1 = evaluate_model(model1, x_test_new1, y_test)

x_test_new2 = x_test[['idade', 'gênero2', 'imc', 'fuma']]
acc2, mape2 = evaluate_model(model2, x_test_new2, y_test)

acc3, mape3 = evaluate_model(model3, x_test, y_test)

In [52]:
print(f"Modelo 1 - Acurácia: {acc1:.2f}%, MAPE: {mape1:.2f}%")
print(f"Modelo 2 - Acurácia: {acc2:.2f}%, MAPE: {mape2:.2f}%")
print(f"Modelo 3 - Acurácia: {acc3:.2f}%, MAPE: {mape3:.2f}%")

Modelo 1 - Acurácia: 30.90%, MAPE: 117.08%
Modelo 2 - Acurácia: 30.62%, MAPE: 120.62%
Modelo 3 - Acurácia: 67.05%, MAPE: 49.32%
